In [33]:
#stolen from https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277
import os
import sys
import random
from collections import OrderedDict
import itertools
import math
import logging
import json
import re

import cv2
import numpy as np
import pandas as pd
import utils
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input, AlphaDropout, Activation, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adamax, Adam, SGD
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = 'stage1_train/'
TEST_PATH = 'stage2_test/'

In [34]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [35]:
# initializes the training images and target images (masks) as arrays of zeros
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
#gets and resizes the images each containing several nuclei
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = cv2.imread(path + '/images/' + id_ + '.png')[:,:,:3]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    #gets and resizes the masks that contain the individual segmented nuclei
    #the targets for the model
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = cv2.imread(path + '/masks/' + mask_file,0)
        mask_ = np.expand_dims(resize(mask_, (256, 256), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# initializes, gets, and resizes test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
sizes_test = []
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    print(n)
    #print(os.path.join(path, '/images/', id_ + '.png'))
    img = cv2.imread(path + '/images/' + id_ + '.png')[:,:,:3]
    #print(img.shape)
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

  0%|          | 7/3019 [00:00<00:45, 66.92it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12


  1%|          | 20/3019 [00:00<00:48, 62.04it/s]

13
14
15
16
17
18
19
20
21
22
23
24


  1%|          | 35/3019 [00:00<00:45, 65.13it/s]

...

2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992


100%|█████████▉| 3006/3019 [00:46<00:00, 64.81it/s]

2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006


100%|██████████| 3019/3019 [00:46<00:00, 64.79it/s]

3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
Done!


In [36]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.1, random_state=8011)

print(len(x_train), len(y_train), len(x_valid), len(y_valid))
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

597 597 67 67
(597, 256, 256, 3) (597, 256, 256, 1) (67, 256, 256, 3) (67, 256, 256, 1)


In [37]:
from keras.losses import binary_crossentropy

# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [38]:
from keras.layers import Layer


class MaxPoolingWithArgmax2D(Layer):
    def __init__(self, pool_size=(2, 2), strides=(2, 2), padding='same', **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs, **kwargs):
        padding = self.padding
        pool_size = self.pool_size
        strides = self.strides
        if K.backend() == 'tensorflow':
            ksize = [1, pool_size[0], pool_size[1], 1]
            padding = padding.upper()
            strides = [1, strides[0], strides[1], 1]
            output, argmax = K.tf.nn.max_pool_with_argmax(inputs, ksize=ksize, strides=strides, padding=padding)
        else:
            errmsg = '{} backend is not supported for layer {}'.format(K.backend(), type(self).__name__)
            raise NotImplementedError(errmsg)
        argmax = K.cast(argmax, K.floatx())
        return [output, argmax]

    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [dim // ratio[idx] if dim is not None else None for idx, dim in enumerate(input_shape)]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]


class MaxUnpooling2D(Layer):
    def __init__(self, size=(2, 2), **kwargs):
        super(MaxUnpooling2D, self).__init__(**kwargs)
        self.size = size

    def call(self, inputs, output_shape=None):
        updates, mask = inputs[0], inputs[1]
        with K.tf.variable_scope(self.name):
            mask = K.cast(mask, 'int32')
            input_shape = K.tf.shape(updates, out_type='int32')
            #  calculation new shape
            if output_shape is None:
                output_shape = (input_shape[0], input_shape[1] * self.size[0], input_shape[2] * self.size[1], input_shape[3])
            self.output_shape1 = output_shape

            # calculation indices for batch, height, width and feature maps
            one_like_mask = K.ones_like(mask, dtype='int32')
            batch_shape = K.concatenate([[input_shape[0]], [1], [1], [1]], axis=0)
            batch_range = K.reshape(K.tf.range(output_shape[0], dtype='int32'), shape=batch_shape)
            b = one_like_mask * batch_range
            y = mask // (output_shape[2] * output_shape[3])
            x = (mask // output_shape[3]) % output_shape[2]
            feature_range = K.tf.range(output_shape[3], dtype='int32')
            f = one_like_mask * feature_range

            # transpose indices & reshape update values to one dimension
            updates_size = K.tf.size(updates)
            indices = K.transpose(K.reshape(K.stack([b, y, x, f]), [4, updates_size]))
            values = K.reshape(updates, [updates_size])
            ret = K.tf.scatter_nd(indices, values, output_shape)
            return ret

    def compute_output_shape(self, input_shape):
        mask_shape = input_shape[1]
        return mask_shape[0], mask_shape[1] * self.size[0], mask_shape[2] * self.size[1], mask_shape[3]

In [39]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, 3))
kernel = 3
pool_size=(2,2)

# encoder
conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
conv_1 = BatchNormalization()(conv_1)
conv_1 = Activation("relu")(conv_1)
conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
conv_2 = BatchNormalization()(conv_2)
conv_2 = Activation("relu")(conv_2)

pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
conv_3 = BatchNormalization()(conv_3)
conv_3 = Activation("relu")(conv_3)
conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
conv_4 = BatchNormalization()(conv_4)
conv_4 = Activation("relu")(conv_4)

pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
conv_5 = BatchNormalization()(conv_5)
conv_5 = Activation("relu")(conv_5)
conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
conv_6 = BatchNormalization()(conv_6)
conv_6 = Activation("relu")(conv_6)
conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
conv_7 = BatchNormalization()(conv_7)
conv_7 = Activation("relu")(conv_7)

pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
conv_8 = BatchNormalization()(conv_8)
conv_8 = Activation("relu")(conv_8)
conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
conv_9 = BatchNormalization()(conv_9)
conv_9 = Activation("relu")(conv_9)
conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
conv_10 = BatchNormalization()(conv_10)
conv_10 = Activation("relu")(conv_10)

pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
conv_11 = BatchNormalization()(conv_11)
conv_11 = Activation("relu")(conv_11)
conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
conv_12 = BatchNormalization()(conv_12)
conv_12 = Activation("relu")(conv_12)
conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
conv_13 = BatchNormalization()(conv_13)
conv_13 = Activation("relu")(conv_13)

pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
print("Build enceder done..")

# between encoder and decoder
conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(pool_5)
conv_14 = BatchNormalization()(conv_14)
conv_14 = Activation("relu")(conv_14)
conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
conv_15 = BatchNormalization()(conv_15)
conv_15 = Activation("relu")(conv_15)
conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
conv_16 = BatchNormalization()(conv_16)
conv_16 = Activation("relu")(conv_16)

# decoder
unpool_1 = MaxUnpooling2D(pool_size)([conv_16, mask_5])
concat_1 = Concatenate()([unpool_1, conv_13])

conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(concat_1)
conv_17 = BatchNormalization()(conv_17)
conv_17 = Activation("relu")(conv_17)
conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
conv_18 = BatchNormalization()(conv_18)
conv_18 = Activation("relu")(conv_18)
conv_19 = Convolution2D(512, (kernel, kernel), padding="same")(conv_18)
conv_19 = BatchNormalization()(conv_19)
conv_19 = Activation("relu")(conv_19)

unpool_2 = MaxUnpooling2D(pool_size)([conv_19, mask_4])
concat_2 = concatenate([unpool_2, conv_10])

conv_20 = Convolution2D(512, (kernel, kernel), padding="same")(concat_2)
conv_20 = BatchNormalization()(conv_20)
conv_20 = Activation("relu")(conv_20)
conv_21 = Convolution2D(512, (kernel, kernel), padding="same")(conv_20)
conv_21 = BatchNormalization()(conv_21)
conv_21 = Activation("relu")(conv_21)
conv_22 = Convolution2D(256, (kernel, kernel), padding="same")(conv_21)
conv_22 = BatchNormalization()(conv_22)
conv_22 = Activation("relu")(conv_22)

unpool_3 = MaxUnpooling2D(pool_size)([conv_22, mask_3])
concat_3 = concatenate([unpool_3, conv_7])

conv_23 = Convolution2D(256, (kernel, kernel), padding="same")(concat_3)
conv_23 = BatchNormalization()(conv_23)
conv_23 = Activation("relu")(conv_23)
conv_24 = Convolution2D(256, (kernel, kernel), padding="same")(conv_23)
conv_24 = BatchNormalization()(conv_24)
conv_24 = Activation("relu")(conv_24)
conv_25 = Convolution2D(128, (kernel, kernel), padding="same")(conv_24)
conv_25 = BatchNormalization()(conv_25)
conv_25 = Activation("relu")(conv_25)

unpool_4 = MaxUnpooling2D(pool_size)([conv_25, mask_2])
concat_4 = concatenate([unpool_4, conv_4])

conv_26 = Convolution2D(128, (kernel, kernel), padding="same")(concat_4)
conv_26 = BatchNormalization()(conv_26)
conv_26 = Activation("relu")(conv_26)
conv_27 = Convolution2D(64, (kernel, kernel), padding="same")(conv_26)
conv_27 = BatchNormalization()(conv_27)
conv_27 = Activation("relu")(conv_27)

unpool_5 = MaxUnpooling2D(pool_size)([conv_27, mask_1])
concat_5 = concatenate([unpool_5, conv_2])

conv_28 = Convolution2D(64, (kernel, kernel), padding="same")(concat_5)
conv_28 = BatchNormalization()(conv_28)
conv_28 = Activation("relu")(conv_28)

outputs = Convolution2D(1, (1, 1), activation='sigmoid')(conv_28)
#conv_29 = BatchNormalization()(conv_29)
#conv_29 = Reshape((input_shape[0] * input_shape[1], n_labels), input_shape=(input_shape[0], input_shape[1], n_labels))(conv_29)

#outputs = Activation(output_mode)(conv_29)
print("Build decoder done..")

segunet = Model(inputs=inputs, outputs=outputs, name="SegUNet")
segunet.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
segunet.summary()

Build enceder done..
Build decoder done..
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_30 (Conv2D)              (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 256, 256, 64) 256         conv2d_30[0][0]                  
__________________________________________________________________________________________________
activation_29 (Activation)      (None, 256, 256, 64) 0           batch_normalization_29[0][0]     
___________________________________________________________________

In [40]:
"""
from keras.preprocessing.image import ImageDataGenerator

data_gen_args = dict(horizontal_flip=True,
                         vertical_flip=True,
                         rotation_range=90.,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
image_datagen.fit(x_train, seed=7)
mask_datagen.fit(y_train, seed=7)
image_generator = image_datagen.flow(x_train, batch_size=32, seed=7)
mask_generator = mask_datagen.flow(y_train, batch_size=32, seed=7)
train_generator = zip(image_generator, mask_generator)

val_gen_args = dict()
image_datagen_val = ImageDataGenerator(**val_gen_args)
mask_datagen_val = ImageDataGenerator(**val_gen_args)
image_datagen_val.fit(x_valid, seed=7)
mask_datagen_val.fit(y_valid, seed=7)
image_generator_val = image_datagen_val.flow(x_valid, batch_size=32, seed=7)
mask_generator_val = mask_datagen_val.flow(y_valid, batch_size=32, seed=7)
valid_generator=zip(image_generator_val, mask_generator_val)
"""

'\nfrom keras.preprocessing.image import ImageDataGenerator\n\ndata_gen_args = dict(horizontal_flip=True,\n                         vertical_flip=True,\n                         rotation_range=90.,\n                         width_shift_range=0.1,\n                         height_shift_range=0.1,\n                         zoom_range=0.1)\nimage_datagen = ImageDataGenerator(**data_gen_args)\nmask_datagen = ImageDataGenerator(**data_gen_args)\nimage_datagen.fit(x_train, seed=7)\nmask_datagen.fit(y_train, seed=7)\nimage_generator = image_datagen.flow(x_train, batch_size=32, seed=7)\nmask_generator = mask_datagen.flow(y_train, batch_size=32, seed=7)\ntrain_generator = zip(image_generator, mask_generator)\n\nval_gen_args = dict()\nimage_datagen_val = ImageDataGenerator(**val_gen_args)\nmask_datagen_val = ImageDataGenerator(**val_gen_args)\nimage_datagen_val.fit(x_valid, seed=7)\nmask_datagen_val.fit(y_valid, seed=7)\nimage_generator_val = image_datagen_val.flow(x_valid, batch_size=32, seed=7

In [41]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
segunet.fit(x_train, y_train, batch_size=16, epochs = 50, validation_data=(x_valid, y_valid), callbacks = [checkpoint])

Train on 597 samples, validate on 67 samples
Epoch 1/50
597/597 [==============================] - 138s 230ms/step - loss: 0.3561 - mean_iou: 0.6184 - val_loss: 0.4523 - val_mean_iou: 0.6564

Epoch 00001: val_loss improved from inf to 0.45231, saving model to best_model.h5
Epoch 2/50
597/597 [==============================] - 115s 193ms/step - loss: 0.2445 - mean_iou: 0.6579 - val_loss: 0.7390 - val_mean_iou: 0.6705

Epoch 00002: val_loss did not improve
Epoch 3/50
597/597 [==============================] - 116s 194ms/step - loss: 0.1669 - mean_iou: 0.6830 - val_loss: 0.2460 - val_mean_iou: 0.7003

Epoch 00003: val_loss improved from 0.45231 to 0.24598, saving model to best_model.h5
Epoch 4/50
597/597 [==============================] - 116s 194ms/step - loss: 0.1241 - mean_iou: 0.7157 - val_loss: 0.2105 - val_mean_iou: 0.7279

Epoch 00004: val_loss improved from 0.24598 to 0.21046, saving model to best_model.h5
Epoch 5/50
597/597 [==============================] - 115s 193ms/step - los


Epoch 00042: val_loss did not improve
Epoch 43/50
597/597 [==============================] - 115s 193ms/step - loss: 0.0673 - mean_iou: 0.8316 - val_loss: 0.0603 - val_mean_iou: 0.8321

Epoch 00043: val_loss did not improve
Epoch 44/50
597/597 [==============================] - 116s 194ms/step - loss: 0.0651 - mean_iou: 0.8325 - val_loss: 0.0621 - val_mean_iou: 0.8330

Epoch 00044: val_loss did not improve
Epoch 45/50
597/597 [==============================] - 115s 193ms/step - loss: 0.0656 - mean_iou: 0.8335 - val_loss: 0.0595 - val_mean_iou: 0.8339

Epoch 00045: val_loss did not improve
Epoch 46/50
597/597 [==============================] - 115s 193ms/step - loss: 0.0651 - mean_iou: 0.8344 - val_loss: 0.0588 - val_mean_iou: 0.8348

Epoch 00046: val_loss did not improve
Epoch 47/50
597/597 [==============================] - 115s 193ms/step - loss: 0.0740 - mean_iou: 0.8352 - val_loss: 0.2213 - val_mean_iou: 0.8353

Epoch 00047: val_loss did not improve
Epoch 48/50
597/597 [==========

In [42]:
# Predict on train, val and test
model = load_model('best_model.h5', custom_objects={'mean_iou': mean_iou, 'MaxPoolingWithArgmax2D':MaxPoolingWithArgmax2D,
                                                   'MaxUnpooling2D':MaxUnpooling2D})
preds_train = model.predict(x_train, verbose=1)
preds_val = model.predict(x_valid, verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))
    
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))
    


3019/3019 [==============================] - 277s 92ms/step


In [43]:
# Create submission DataFrame

sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.groupby('ImageId').ngroup()
print(len(sub))
sub.to_csv('SegUnet_no_aug.csv', index=False)

131366
